In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
import random

In [8]:
!pwd

/data/sulixin/research/contest/race/RACE


In [19]:
def merge_file(datadir, outfile):
    rv = {'data': []}
    
    for f in os.listdir(datadir):
        if 'txt' in f:
            rv['data'].append(json.loads(open(os.path.join(datadir, f)).read()))
    json.dump(rv,open(outfile, 'w'),indent=2)
for level in ['middle', 'high']:
    for split in ['train', 'dev', 'test']:     
        merge_file('/data/sulixin/research/contest/race/RACE/{}/{}'.format(split, level), 
           '/data/sulixin/research/contest/race/RACE/{}_{}.json'.format(split, level))

In [22]:
!head -n50 train_high.json

{
  "data": [
    {
      "answers": [
        "C",
        "C",
        "D",
        "B"
      ],
      "options": [
        [
          "doctor",
          "model",
          "teacher",
          "reporter"
        ],
        [
          "marry a better man/woman",
          "become a model",
          "get an advantage over others in job-hunting",
          "attract more admirers"
        ],
        [
          "everyone should purchase perfection, whatever the cost",
          "it's right for graduates to ask for others to help them out in hunting for jobs",
          "it is one's appearance instead of skills that really matters in one's career",
          "media are to blame for misleading young people in their seeking for surgery"
        ],
        [
          "Young Graduates Have Higher Expectations",
          "Young Graduates Look to Surgery for Better Jobs",
          "Young Graduates' Opinion About Cosmetic Surgery",
          "Young Graduates Face a Different Situation in

In [25]:
def my_load(filename):
    data = json.load(open(filename))['data']
    print('process file {} has {} sample'.format(os.path.basename(filename), len(data)))
    return data
train_high = my_load('/data/sulixin/research/contest/race/RACE/train_high.json')
train_middle = my_load('/data/sulixin/research/contest/race/RACE/train_middle.json')
dev_high = my_load('/data/sulixin/research/contest/race/RACE/dev_high.json')
dev_middle = my_load('/data/sulixin/research/contest/race/RACE/dev_middle.json')
test_high = my_load('/data/sulixin/research/contest/race/RACE/test_high.json')
test_middle = my_load('/data/sulixin/research/contest/race/RACE/test_middle.json')

process file train_high.json has 18728 sample
process file train_middle.json has 6409 sample
process file dev_high.json has 1021 sample
process file dev_middle.json has 368 sample
process file test_high.json has 1045 sample
process file test_middle.json has 362 sample


In [30]:
def my_stat_context(data):
    rv = {'context_len': [len(x['article'].split()) for x in data],\
         'qa_cnt': [len(x['answers']) for x in data],
         }
    return pd.DataFrame(rv).describe()

my_stat_context(train_high)

,context_len,qa_cnt
count,18728.000000,18728.000000
mean,307.973943,3.334312
std,82.271533,0.913924
min,3.000000,0.000000
25%,264.000000,3.000000
50%,302.000000,3.000000
75%,341.000000,4.000000
max,1162.000000,6.000000


In [29]:
my_stat_context(train_middle)

,context_len,qa_cnt
count,6409.000000,6409.000000
mean,197.824622,3.966453
std,73.920773,0.996074
min,2.000000,1.000000
25%,142.000000,3.000000
50%,195.000000,4.000000
75%,247.000000,5.000000
max,530.000000,7.000000


In [37]:
def iter_qa(data):
    for d in data:
        for q,a,o in zip(d['questions'],d['answers'], d['options']):
            yield q,a,o

def word_len(text):
    return len(text.split())
    
def avg(li):
    return sum(li) * 1.0 / len(li)
    
def my_stat_qa(data):
    rv = {'answer': [x[1] for x in iter_qa(data)],\
         'q_len': [word_len(x[0]) for x in iter_qa(data)],\
         'op_len': [avg([word_len(y) for y in x[-1]]) for x in iter_qa(data)],\
         'op_cnt':  [len(x[-1]) for x in iter_qa(data)]
         }
    df = pd.DataFrame(rv)
    print(df['answer'].value_counts())
    return df.describe()

my_stat_qa(train_high)

C    16885
B    16021
D    15838
A    13701
Name: answer, dtype: int64


,q_len,op_len,op_cnt
count,62445.000000,62445.000000,62445.0
mean,10.400929,6.234146,4.0
std,3.327790,3.145362,0.0
min,1.000000,1.000000,4.0
25%,8.000000,4.000000,4.0
50%,10.000000,6.250000,4.0
75%,12.000000,8.250000,4.0
max,63.000000,70.000000,4.0


In [38]:
my_stat_qa(train_middle)

C    7006
B    6705
D    6265
A    5445
Name: answer, dtype: int64


,q_len,op_len,op_cnt
count,25421.000000,25421.000000,25421.0
mean,9.045514,4.408304,4.0
std,3.320716,2.894599,0.0
min,1.000000,1.000000,4.0
25%,7.000000,2.000000,4.0
50%,9.000000,3.750000,4.0
75%,11.000000,6.500000,4.0
max,63.000000,22.750000,4.0
